# Implementation of Revised Knuth–Morris–Pratt Searching Algorithm on Genome Sequence
Note: Run all cells in this notebook in order

### 1. File Handling
>#### Functions to read in and clean genome (DNA/RNA) sequence
>Note: Data cleaning function works with .fna files from https://www.ncbi.nlm.nih.gov/genome/

In [1]:
# To read .fna file

def readSourceFile(filename):
    source_file = open(filename, 'r')
    file = source_file.read()
    source_file.close()
    return file

In [2]:
# to get rid of title and newline characters in the .fna file
# return a clean genome sequence

def cleanSourceFile(filename):
    source_file = open(filename,'r')
    lines = source_file.readlines()[1:] # to remove title
    source_file.close()
    raw_DNA = "".join(lines)
    clean_DNA = raw_DNA.replace('\n','') # to remove newline characters
    clean_DNA = clean_DNA.upper()
    return clean_DNA


# This function is only applicable to .fna files downloaded from https://www.ncbi.nlm.nih.gov/genome/
# .fna files from other resources may have different formats, thus this function needs to be varied accordingly

### 2. Revised KMP Searching Algorithm
Contains:
- Prepocessing functions in intialisation (Construction of bad match table and lps array)  
- Search Function  


In [3]:
# Note: lps stands for longest proper prefix that is also a proper suffix

def computeLPSarray(query, M, lps): # M represents length of query sequence
    ''' Preprocessing - Constructs LPS array from query sequence '''
    len = 0 # pointer of length of the matching proper prefix that is also a suffix
    i = 1 # pointer variable to traverse the query sequence
    lps[0] = 0
    
    while i < M:
        if query[i] == query[len]: # a matched character found
            lps[i] = len + 1 # store the length of the matching prefix/suffix
            len += 1
            i += 1 # increase both pointers by 1
        else: # characters do not match
            if len != 0: 
                len = lps[len-1] # to maintain continuity
            else:
                lps[i] = 0
                i += 1

In [4]:
def badheu(pattern,bad_match_table,pattern_len,alpha):
    ''' Preprocessing - Constructs bad match table from query sequence '''
    
    # Creating {"A":0, "C":1, "G":2, "T/U":3} for "ACGT"/"ACGU"
    if alpha==1:
        alphabet_map = {"A":0, "C":1, "G":2, "T":3}
    else:
        alphabet_map = {"A":0, "C":1, "G":2, "U":3}
      
    # Creating bad_match_table
    # Eg. "AGCTTC", alphabet_map = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    #  A  C  G  T
    # [5, 3, 4, 1] //bad match table created

    # if it is the last character of the patten and:
    # a) it has not already been defined -> leave it to be pattern_len
    # b) it had been defined -> leave it to be the value of last matching character
    for i in range(pattern_len-1):
        char_to_index = alphabet_map[pattern[i]]
        bad_match_table[char_to_index] = pattern_len - i - 1 # All other characters = pattern_len - greatest_index_of_this_char - 1

In [5]:
def KMPModi(query, source, alpha):
    ''' Searching - make use of preprocessed array and table to search for query sequence '''
    
    # create an empty list, for storing positions of matches found later
    occurrences = [] 
    
    # store the length of source genome and query sequence in variable M, N respectively
    N = len(source)  
    M = len(query)
    
    lastPat = query[M-1] #obtaining last character from pattern
    
    # Initialize the lps array of length M with each entry to be 0
    lps = [0] * M
    # Call function to compute lps array
    computeLPSarray(query, M, lps)

    # Creating table for bad char heuristic   
    # All alphabets that do not appear in the pattern are set to len(pattern) since no other parts of the pattern would match 
    bad_match_table = [M] * 4 # creates[pattern_len, pattern_len, pattern_len, pattern_len]
    badheu(query,bad_match_table,M,alpha) # updating table
    if alpha==1:
        alphabet_map = {"A":0, "C":1, "G":2, "T":3}
    else:
        alphabet_map = {"A":0, "C":1, "G":2, "U":3}

    # Searching process
    i = 0 # pointer variable for source sequence
    j = 0 # pointer variable for query sequence
    
    while i < N-M-1:
        if source[i] == query[j]: # character matched
            i += 1 # query pointer increase by 1
            j += 1 # pattern pointer increase by 1
        else:
            if j != 0: #mismatch does not occur at first character of query sequence
                lastTxt=source[i+M-lps[j-1]-1] #obtaining character of source which position matches that of last character of query after jumping based on lps table
                if (lastTxt!=lastPat): #if character of source (which position matches that of last character of query after jumping) does not match last char of query 
                    j=0 #set query pointer to 0
                    i=i+bad_match_table[alphabet_map[lastTxt]]-lps[j-1] #increase source pointer to decrease comparisons
                else: #if character of source (which position matches that of last character of query after jumping) matches last char of query
                    j = lps[j-1]# update new starting index for query sequence with lps table
            else:# when mismatch occurs in first character of query sequence
                i += 1# increase i by 1 to check the next character in source sequence
        if j == M: # pointer traverses to the end of the query sequence successfully, match found
            occurrences.append(i-j+1) # return index position of found match, starting from position 1 instead of 0
            j = lps[j-1] # reset j to continue searching
    print("The DNA segment can be found at the position(s) of", occurrences,", there are/is", len(occurrences),"occurrence(s) in total.")

### 3. Execution
#### Searching UI starts here
_Make sure sequence does not contain any other alphabet other than "ACGT" for DNA or "ACGU" for RNA for example "N"_

In [6]:
# To read in.fna file and get type of the file, DNA or RNA
file = input("Please enter the source file.fna: ")
type_of_sequence = input("Please input type of sequence. (1) DNA, (2) RNA (key in 1 or 2): ")

# print the content of the .fna file
print("The input file: ")
file_read = readSourceFile(file)
file_read

Please enter the source file.fna: test.fna
Please input type of sequence. (1) DNA, (2) RNA (key in 1 or 2): 1
The input file: 


'>NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome\...\nAAAAAAAAAAAAA'

In [7]:
# print the clean genome sequence
print("Complete genome sequence after cleaning: ")
source = cleanSourceFile(file)
source

Complete genome sequence after cleaning: 


'...'

In [8]:
# prompt the user to enter a query sequence
target = input("Please enter the query sequence to be found: ")

if (type_of_sequence == str(1)):
    alpha = 1 #DNA
elif (type_of_sequence == str(2)):
    alpha = 2 #RNA

import datetime
run_start_time = datetime.datetime.now() 

# print results
KMPModi(target, source, alpha)

# to return processing time of the searching algorithm
run_time = datetime.datetime.now() - run_start_time
print("Search took", run_time.microseconds, "microseconds")

Please enter the query sequence to be found: CTCTTGAAACTGCTCAAAATTCTG
The DNA segment can be found at the position(s) of [1917] , there are/is 1 occurrence(s) in total.
Search took 7977 microseconds


### 4. References
- https://www.geeksforgeeks.org/kmp-algorithm-for-pattern-searching/  
- https://www.youtube.com/watch?v=4jY57Ehc14Y  
- https://www.cnki.net/kcms/detail/11.2127.TP.20141211.1526.043.html